**Goal**: Introduce basic SQL structure and show some example data transformations. Understand CTEs, aliases, and `CASE` statements. Preview boolean logic, `COALESCE`, `NOT NULL`, and other forms of filtering.

In [2]:
import duckdb

# Load SQL extension
%load_ext sql

# Initialize 🦆 DuckDB connection
conn = duckdb.connect()

# Import database
%sql conn --alias duckdb
%sql IMPORT DATABASE '../../data/nps';

Loading configurations from /workspaces/fundamentals-of-data-transformation-4554074/pyproject.toml.

Settings changed:

Config,value
feedback,True
autopandas,True
displaylimit,10
displaycon,False


,Count
0,224


We often deal with a mix of structured and semi-structured data in SQL transformations, let's see what our parks dataset looks like.

In [3]:
%%sql
SELECT * FROM nps_public_data.parks LIMIT 5


,relevanceScore,designation,weatherInfo,addresses,operatingHours,entrancePasses,name,description,directionsUrl,fees,...,activities,url,longitude,id,images,directionsInfo,fullName,parkCode,latLong,latitude
0,1,National Memorial,http://forecast.weather.gov/MapClick.php?CityN...,"[{'type': 'Physical', 'line2': '', 'line1': '1...","[{'name': 'Hours of Operation', 'standardHours...",[],Federal Hall,"Here on Wall Street, George Washington took th...",http://www.nps.gov/feha/planyourvisit/directio...,[],...,"[{'name': 'Arts and Culture', 'id': '09DF0950-...",https://www.nps.gov/feha/index.htm,-74.010256,2337D255-2D32-4997-957A-D461EEA03AF8,[{'url': 'https://www.nps.gov/common/uploads/s...,The main entrance of Federal Hall is located a...,Federal Hall National Memorial,feha,"lat:40.70731192, long:-74.01025636",40.707312
1,1,National Historic Trail,"In winter, watch for ice on trails and sidewal...","[{'type': 'Physical', 'line2': '', 'line1': '6...","[{'name': 'Visitor Center Hours', 'standardHou...",[],Lewis & Clark,The Lewis and Clark National Historic Trail wi...,https://www.nps.gov/lecl/,[],...,"[{'name': 'Auto and ATV', 'id': '5F723BAD-7359...",https://www.nps.gov/lecl/index.htm,-95.924515,5D443C5F-19A0-4A06-9CE4-30534A3DD81A,[{'url': 'https://www.nps.gov/common/uploads/s...,Lewis & Clark National Historic Trail Headquar...,Lewis & Clark National Historic Trail,lecl,"lat:41.2646141052, long:-95.9245147705",41.264614
2,1,,"Summers are generally hot and humid, with dayt...","[{'type': 'Physical', 'line2': '', 'line1': '1...",[{'name': 'National Capital Parks-East Headqua...,[],National Capital Parks-East,Welcome to National Capital Parks-East. We inv...,http://www.nps.gov/nace/planyourvisit/directio...,[],...,"[{'name': 'Biking', 'id': '7CE6E935-F839-4FEC-...",https://www.nps.gov/nace/index.htm,-76.994000,BA3C1A1D-AA6A-49EB-9237-0222CEEE2670,[{'url': 'https://www.nps.gov/common/uploads/s...,DC295 South to the Exit for I-694/I-395/Capito...,National Capital Parks-East,nace,"lat:38.8659, long:-76.994",38.865900
3,1,National Historical Park,"Be prepared for hot, humid weather. The histor...","[{'type': 'Physical', 'line2': '', 'line1': '1...","[{'name': 'Visitor Center', 'standardHours': {...",[{'description': 'Adams National Historical Pa...,Adams,From the sweet little farm at the foot of Penn...,http://www.nps.gov/adam/planyourvisit/directio...,[],...,"[{'name': 'Guided Tours', 'id': 'B33DC9B6-0B7D...",https://www.nps.gov/adam/index.htm,-71.011604,E4C7784E-66A0-4D44-87D0-3E072F5FEF43,[{'url': 'https://www.nps.gov/common/uploads/s...,"Traveling on U.S. Interstate 93, take exit 7 -...",Adams National Historical Park,adam,"lat:42.2553961, long:-71.01160356",42.255396
4,1,Memorial Parkway,Summers on the parkway are generally hot and h...,"[{'type': 'Physical', 'line2': '700 George Was...",[{'name': 'George Washington Memorial Parkway ...,[],George Washington,The George Washington Memorial Parkway was des...,http://www.nps.gov/gwmp/planyourvisit/directio...,[],...,"[{'name': 'Arts and Culture', 'id': '09DF0950-...",https://www.nps.gov/gwmp/index.htm,-77.149500,E6D5BB41-3251-469F-ABDA-7B43B966F0CF,[{'url': 'https://www.nps.gov/common/uploads/s...,Directions to Parkway Headquarters From the so...,George Washington Memorial Parkway,gwmp,"lat:38.9628, long:-77.1495",38.962800


Note the type of `operatingHours`— `STRUCT`! That means it's a list or JSON.

In [4]:
%%sql
-- Callout: query structuring, LIMIT statements
SELECT 
    name, 
    operatingHours as operating_hours
FROM nps_public_data.parks 
LIMIT 1

,name,operating_hours
0,Federal Hall,"[{'name': 'Hours of Operation', 'standardHours..."


What if we want to create an `operatingHours` table? We can unpack `json` using `UNNEST`. Notice what we're doing here: there are two operations happening, but we're splitting them up! That's called a CTE (common table expression). It's a way of separating aggregates or other operations.

Next, we're using `UNNEST` to explode the `STRUCT` or `json` data. DuckDB let's us use `recursive := true` to burrow down and get _every_ level of the `json`... Pretty neat!

In [14]:
%%sql
-- Callout: CTEs, UNNEST
WITH park_hours AS (
    SELECT 
        name as park_name, 
        id as park_id, 
        UNNEST(operatingHours, recursive := true)
    FROM nps_public_data.parks
)
SELECT 
    * EXCLUDE (exceptions, name),
    name as category
FROM park_hours
LIMIT 3


,park_name,park_id,friday,sunday,thursday,tuesday,saturday,monday,wednesday,description,category
0,Federal Hall,2337D255-2D32-4997-957A-D461EEA03AF8,10:00AM - 5:00PM,Closed,10:00AM - 5:00PM,10:00AM - 5:00PM,Closed,10:00AM - 5:00PM,10:00AM - 5:00PM,Federal Hall is Open.,Hours of Operation
1,Lewis & Clark,5D443C5F-19A0-4A06-9CE4-30534A3DD81A,8:30AM - 4:30PM,Closed,8:30AM - 4:30PM,8:30AM - 4:30PM,Closed,8:30AM - 4:30PM,8:30AM - 4:30PM,Lewis and Clark National Historic Trail Visito...,Visitor Center Hours
2,National Capital Parks-East,BA3C1A1D-AA6A-49EB-9237-0222CEEE2670,8:00AM - 4:00PM,Closed,8:00AM - 4:00PM,8:00AM - 4:00PM,Closed,8:00AM - 4:00PM,8:00AM - 4:00PM,National Capital Parks-East is an administrati...,National Capital Parks-East Headquarters


Notice how we use a CTE to make the query easy-to-read and logical. Now we can create a table with the result. `EXCLUDE` lets us use `SELECT *` and remove some unnecessary inclusions.

In [6]:
%%sql
-- Callout: column renaming, EXCLUDE
CREATE OR REPLACE TABLE nps_public_data.park_hours AS (
    WITH park_hours AS (
        SELECT 
            name as park_name, 
            id as park_id, 
            -- https://duckdb.org/docs/sql/query_syntax/unnest.html
            UNNEST(operatingHours, recursive := true)
        FROM nps_public_data.parks
    )
    SELECT 
        * EXCLUDE (exceptions, name),
        name as category
    FROM park_hours 
)

,Count
0,667


Creating tables with _dimensions_, like operating hours, lets us easily join to access the information. Here, notice how readable the query becomes.

We're selecting the _name_ of the park and the _thursday_ hours `WHERE` the category is 'Hours of Operation'.

If we'd included the above logic in this query, it'd be much more dense! This is one of our first _patterns_ for SQL transformation:

> Store precalculated (or aggregated) queries or use CTEs to limit complexity and improve readability

In [7]:
%%sql
-- Callout: WHERE clause
SELECT
    p.name,
    h.thursday
FROM nps_public_data.park_hours h
LEFT JOIN nps_public_data.parks p
    ON h.park_id = p.id
WHERE h.category = 'Hours of Operation'
LIMIT 5

,name,thursday
0,Federal Hall,10:00AM - 5:00PM
1,Theodore Roosevelt Birthplace,10:00AM - 4:00PM
2,Tumacácori,9:00AM - 5:00PM
3,Wright Brothers,9:00AM - 5:00PM


**Note:** It's important to use single quotes in DuckDB SQL ('), double quotes (") are reserved for table names

If we want to know all the values that `thursday` can take, we can use `DISTINCT` to return a list... This is like using `set()` in Python.

In [8]:
%%sql 
# Callout: DISTINCT, Order, LIMIT
SELECT 
    DISTINCT(thursday) 
FROM nps_public_data.park_hours 
ORDER BY 1 DESC 
LIMIT 10;

,thursday
0,unknown
1,Sunrise to Sunset
2,Opens at 6:00AM
3,Opens at 5:00AM
4,Closes at 12:00PM
5,Closed
6,All Day
7,9:30AM - 5:00PM
8,9:30AM - 4:30PM
9,9:30AM - 4:00PM


We can use `CASE` functions to alter how data is returned or create entirely new columns. Here, we'll create a new table, renaming columns as we go. This is an example of _cleaning_ a dataset. We'll assume 'unknown' hours are closed park resources.

In [9]:
%%sql
CREATE OR REPLACE TABLE nps_public_data.park_hours AS (
    WITH park_hours AS (
        SELECT 
            name as park_name, 
            id as park_id, 
            -- https://duckdb.org/docs/sql/query_syntax/unnest.html
            UNNEST(operatingHours, recursive := true)
        FROM nps_public_data.parks
    )
    SELECT 
        park_name,
        park_id,
        description,
        name as category,
        CASE monday WHEN 'unknown' THEN 'Closed' ELSE monday END as monday_hours,
        CASE tuesday WHEN 'unknown' THEN 'Closed' ELSE tuesday END as tuesday_hours,
        CASE wednesday WHEN 'unknown' THEN 'Closed' ELSE wednesday END as wednesday_hours,
        CASE thursday WHEN 'unknown' THEN 'Closed' ELSE thursday END as thursday_hours,
        CASE friday WHEN 'unknown' THEN 'Closed' ELSE friday END as friday_hours,
        CASE saturday WHEN 'unknown' THEN 'Closed' ELSE saturday END as saturday_hours,
        CASE sunday WHEN 'unknown' THEN 'Closed' ELSE sunday END as sunday_hours,
        CASE WHEN 
            monday != 'Closed' AND
            tuesday != 'Closed' AND
            wednesday != 'Closed' AND
            thursday != 'Closed' AND
            friday != 'Closed' AND
            saturday != 'Closed' AND
            sunday != 'Closed'
        THEN TRUE ELSE FALSE END as open_seven_days_a_week
    FROM park_hours 
)

,Count
0,667


In the above, we create a boolean column, `open_seven_days_a_week`, that tells us if a park is open every day. Now this might seem repetitive, given that information is already contained in parks, but what it unlocks is a precise, easily readable filter:

In [10]:
%%sql
SELECT * FROM nps_public_data.park_hours WHERE open_seven_days_a_week LIMIT 25

,park_name,park_id,description,category,monday_hours,tuesday_hours,wednesday_hours,thursday_hours,friday_hours,saturday_hours,sunday_hours,open_seven_days_a_week
0,George Washington,E6D5BB41-3251-469F-ABDA-7B43B966F0CF,The George Washington Memorial Parkway is gene...,The George Washington Memorial Parkway,All Day,All Day,All Day,All Day,All Day,All Day,All Day,True
1,Eleanor Roosevelt,04CB5DB1-522B-4C2C-BB3F-C36361E391E7,Park and Val-Kill Cottage tours vary by season...,"Park Grounds, Gardens, and Trails",7:00AM - 5:00PM,7:00AM - 5:00PM,7:00AM - 5:00PM,7:00AM - 5:00PM,7:00AM - 5:00PM,7:00AM - 5:00PM,7:00AM - 5:00PM,True
2,Morristown,D68D9424-CD32-434F-88BF-3069ADFB2744,Current Grounds Hours: 8 am to 5 pm\nIn Morri...,Park Grounds,8:00AM - 5:00PM,8:00AM - 5:00PM,8:00AM - 5:00PM,8:00AM - 5:00PM,8:00AM - 5:00PM,8:00AM - 5:00PM,8:00AM - 5:00PM,True
3,Cedar Breaks,DFC1B0DB-832E-4C21-9CB9-79FB726B97D5,"Visitor access, services, and activities vary ...",Park Hours,All Day,All Day,All Day,All Day,All Day,All Day,All Day,True
4,Devils Postpile,2DC9444C-880F-4738-BA1F-6FB123935A02,"In the operating season, Devils Postpile Natio...",Devils Postpile National Monument,All Day,All Day,All Day,All Day,All Day,All Day,All Day,True
5,Isle Royale,0F6893CF-FC15-4AC5-8C95-E70FC9C21B1A,Isle Royale National Park is open annually fro...,Isle Royale National Park Open Season (April 1...,All Day,All Day,All Day,All Day,All Day,All Day,All Day,True
6,Jean Lafitte,ED57A2B1-2560-471C-82A7-7EF6BC3486F8,Each of Jean Lafitte's six sites is open year-...,Jean Lafitte National Historical Park and Pres...,All Day,All Day,All Day,All Day,All Day,All Day,All Day,True
7,Ice Age,DED290D0-879A-4134-BBC3-54980D80959E,The Ice Age National Scenic Trail is not your ...,Trail,All Day,All Day,All Day,All Day,All Day,All Day,All Day,True
8,Lake Meredith,AC3B46AA-B4C2-4555-A82A-D4CF790DF81D,The National Recreation Area is open 24 hours ...,Park Operating Hours,All Day,All Day,All Day,All Day,All Day,All Day,All Day,True
9,Black Canyon Of The Gunnison,BDBD573F-97EF-44E7-A579-471679F2C42A,"The park is open 24 hours a day, every day, an...",South Rim,All Day,All Day,All Day,All Day,All Day,All Day,All Day,True


As a data or analytics engineer, it's important to make queries as readable as possible. If you know users downstream are often querying on `open_seven_days_a_week`, you can add a similar filter to make everyone's life easier!

Can we find parks that are closed on Thursday?

In [11]:
%%sql
SELECT
    p.name,
    closed_thurs.category,
    closed_thurs.thursday_hours,
    COALESCE(closed_thurs.thursday_hours, 'Open') as closed_open,
    NOT closed_thurs.thursday_hours IS NULL as is_closed
FROM nps_public_data.parks p
INNER JOIN nps_public_data.park_hours closed_thurs
    ON closed_thurs.park_id = p.id
    AND closed_thurs.thursday_hours = 'Closed'
WHERE 1 = 1
ORDER BY RANDOM()
LIMIT 5;

,name,category,thursday_hours,closed_open,is_closed
0,Golden Gate,Battery Townsley,Closed,Closed,True
1,Edgar Allan Poe,Historic site,Closed,Closed,True
2,Ninety Six,Star Fort Pond,Closed,Closed,True
3,Harriet Tubman,Thompson Memorial African Methodist Episcopal ...,Closed,Closed,True
4,Belmont-Paul Women's Equality,Museum,Closed,Closed,True


Notice how we can represent the information in multiple ways— `is_closed`, `closed_open`, and `thursday` all contain the same information, but in different formats. There is no "correct" format— it depends entirely on how you use the data!
- Boolean columns are readable for filters `SELECT * FROM thursday WHERE is_closed`
- Human readable text makes it easier for users to intuit data `SELECT * FROM parks WHERE thursday_hours = 'Closed'`

In [12]:
%%sql
EXPORT DATABASE '../../data/nps' (FORMAT PARQUET, COMPRESSION ZSTD, ROW_GROUP_SIZE 100000);

,Success


Notice the pattern:
- Investigate data
- Identify useful facts/dimensions that could be useful
- Data modelling 
- Transformation
- Storage

A few other things:
1. Data modelling is a useful skill that won't be discussed in this course— for helpful reading see the appendix.
2. _Automating_ transformations is another useful tactic that _also_ won't be discussed in this course. See the appendix for data automations.